In [1]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
debug = False

In [2]:
class Node():
    descendants = []
    MAX = 10000
    
    def __init__(self, name,left, right, value, isMax=True):
        self.name = name
        self.left = left
        self.right = right
        self.alpha = -Node.MAX
        self.beta = Node.MAX
        self.value = value
        self.isMax = isMax
       
    def __str__(self):
        return '('+self.name+','+str(self.alpha)+','+str(self.beta)+','+str(self.left)+','+str(self.right)+','+str(self.value)+','+str(self.isMax)+')'
    
    def isTerminal(self):
        return self.left == None
    
    def children(self):
        print(self.left.name + ',' + self.right.name +'(ligne25)')
        return [{'left' : self.left, 'right' : self.right}]
    
    def __all_children(self):   #private method
        if self.isTerminal():
            #if debug : print(self.name,'terminal')
            return
        else:
            if debug : print(self.name, 'parent')
            self.left.__all_children()
            self.right.__all_children()
            Node.descendants.append({'left' : self.left, 'right' : self.right})
            return Node.descendants
    
    def all_children(self):  
        Node.descendants = []
        return self.__all_children()
    
    def Max_value(self):
        if self.isTerminal(): return self.value
        v = -Node.MAX
        for c in self.children():
            for side in ('left', 'right'):
                w = c[side].Min_value()
                if w > v : v = w
                if w >= self.beta : return v
                if w > self.alpha : self.alpha = w
        return v
                                      
    def Min_value(self):
        if self.isTerminal(): return self.value
        v = Node.MAX
        for c in self.children():
            for side in ('left', 'right'):
                w = c[side].Max_value()
                if w < v : v = w
                if w <= self.alpha : return v
                if w < self.beta : self.beta = w
        return v
    
    def minmax(self):
        if self.isMax:
            v = self.Max_value()
        else:
            v = self.Min_value()
        return v
    
    def alphabeta(self):
        #print('Calling alphabeta()')
        if self.isTerminal(): print('value'); return self.value
        if self.isMax:
            for c in self.children():
                for side in ('left', 'right'):
                    print(self.name, side, '>', c[side].name, end=':')
                    c[side].alpha, c[side].beta = self.alpha, self.beta
                    self.alpha = max(self.alpha, c[side].alphabeta())                    
                    if side =='left' and self.alpha >= self.beta : print(self.name,'right pruning (ligne81)'); break
                    #print(side, c[side].name)
                self.value = self.alpha
            return self.alpha    
        else:
            for c in self.children():
                for side in ('left', 'right'):
                    print(self.name, side, '>', c[side].name, end=':')
                    c[side].alpha, c[side].beta = self.alpha, self.beta
                    self.beta = min(self.beta, c[side].alphabeta())
                    #self.alpha = max(self.alpha,c[side].alpha)
                    if side == 'left' and self.alpha >= self.beta : print(self.name,'right pruning (ligne92)'); break
                    #print(side, c[side].name)
                self.value = self.beta
            return self.beta   
        

In [3]:
class Tree(Node):
    set = []
    depth = 0
    
    def __init__(self,depth):
        self.set = []
        self.depth = depth
        print('Create tree')
        for n in range(depth-1,-1,-1):
            row = []
            ismax = [True, False][n%2]
            for r in range(2**n): 
                name = "nodeL"+str(n)+str(r)
                node = Node(name,None,None,np.random.randint(randMax), ismax)
                #if debug: print(node.name)
                row.append(node)
            if debug: print(row,len(row))
            self.set.append(row)    
        #if debug: print(self.set,len(self.set))
        # connect nodes
        for r in range(1,depth):
            i = 0
            for n in self.set[r]:
                n.left = self.set[r-1][i]
                i += 1
                n.right = self.set[r-1][i]
                i += 1
        print('End create tree\n')
                
    def __str__(self):
        text = ""
        for r in range(len(self.set)):
            for n in self.set[r]:
                text += str(r)+","+str(n.left)+","+str(n.right)+"\t"                
            text += "\n"
        return text
    
    def drawTree(self):
        print('\nDrawing tree')
        for r in range(self.depth):            
            if debug : print(['Node Min','Node Max'][self.set[r][0].isMax])
            for n in self.set[r]:
                if r:     #print((n.alpha,n.beta), end=';')
                    if self.set[r][0].isMax:
                        if debug:
                            print((n.alpha,n.beta), end=';')
                        else:
                            print(n.alpha, end=',')
                    else:
                        if debug:
                            print((n.alpha,n.beta), end=';')
                        else:
                            print(n.beta, end=',')
                else:
                    print(n.value, end=',')
            print()

In [4]:
# test class Node
nodeA  = Node('A',None,None,0)
print(nodeA)
print(nodeA.name)

(A,-10000,10000,None,None,0,True)
A


In [5]:
# test Node methods
print(nodeA.isTerminal())
print('nodeA',nodeA.Max_value())

True
nodeA 0


In [6]:
# building tree
nodeL3  = Node('L3',None,None, np.random.randint(50))
nodeL4  = Node('L4',None,None, np.random.randint(50))
nodeL6  = Node('L6',None,None, np.random.randint(50))
nodeL7  = Node('L7',None,None, np.random.randint(50))
nodeL2  = Node('L2',nodeL3,nodeL4, 0,False)
nodeL5  = Node('L5',nodeL6,nodeL7, 0,False)
nodeL1  = Node('L1',nodeL2,nodeL5,0)

In [7]:
# print attrib
print('Leaf values :',nodeL3.value, nodeL4.value, nodeL6.value, nodeL7.value)
print('nodeL1 left :',nodeL1.left)
print('nodeL1 right :',nodeL1.right)

Leaf values : 44 39 17 12
nodeL1 left : (L2,-10000,10000,(L3,-10000,10000,None,None,44,True),(L4,-10000,10000,None,None,39,True),0,False)
nodeL1 right : (L5,-10000,10000,(L6,-10000,10000,None,None,17,True),(L7,-10000,10000,None,None,12,True),0,False)


In [8]:
# method __str__
print('status nodeL3 :',nodeL3)
print('status nodeL2 :',nodeL2)

status nodeL3 : (L3,-10000,10000,None,None,44,True)
status nodeL2 : (L2,-10000,10000,(L3,-10000,10000,None,None,44,True),(L4,-10000,10000,None,None,39,True),0,False)


In [9]:
# method children()
print('nodeL1 children :',nodeL1.children())
print('nodeL1 left child :',nodeL1.left.children())

L2,L5(ligne25)
nodeL1 children : [{'left': <__main__.Node object at 0x7fd34e3bd290>, 'right': <__main__.Node object at 0x7fd34e3bc190>}]
L3,L4(ligne25)
nodeL1 left child : [{'left': <__main__.Node object at 0x7fd34e378310>, 'right': <__main__.Node object at 0x7fd34e502cd0>}]


In [10]:
# method all_children
print('nodeL1 all children :',nodeL1.all_children())
print('nodeL5 all children :',nodeL5.all_children())

nodeL1 all children : [{'left': <__main__.Node object at 0x7fd34e378310>, 'right': <__main__.Node object at 0x7fd34e502cd0>}, {'left': <__main__.Node object at 0x7fd34e379e50>, 'right': <__main__.Node object at 0x7fd34e3bc0d0>}, {'left': <__main__.Node object at 0x7fd34e3bd290>, 'right': <__main__.Node object at 0x7fd34e3bc190>}]
nodeL5 all children : [{'left': <__main__.Node object at 0x7fd34e379e50>, 'right': <__main__.Node object at 0x7fd34e3bc0d0>}]


In [11]:
# chain call
print('nodeL1',nodeL1.all_children()[0]['left'].name)

nodeL1 L3


In [12]:
# method Max_value()
print('nodeL1 max_value :',nodeL1.Max_value())
# method Min_value()
print('nodeL2 min_value :',nodeL2.Min_value())
# method minmax()
print('nodeL2 minmax :',nodeL2.minmax())

L2,L5(ligne25)
L3,L4(ligne25)
L6,L7(ligne25)
nodeL1 max_value : 39
L3,L4(ligne25)
nodeL2 min_value : 39
L3,L4(ligne25)
nodeL2 minmax : 39


In [13]:
print('nodeL5 min_value :',nodeL5.Min_value())
print('nodeL3 max_value :',nodeL3.Max_value())   # terminal so return value

L6,L7(ligne25)
nodeL5 min_value : 12
nodeL3 max_value : 44


In [14]:
#comparing methods
print('nodeL1 minmax :',nodeL1.minmax())
print('nodeL1 max_value :',nodeL1.Max_value())

L2,L5(ligne25)
L3,L4(ligne25)
L6,L7(ligne25)
nodeL1 minmax : 39
L2,L5(ligne25)
L3,L4(ligne25)
L6,L7(ligne25)
nodeL1 max_value : 39


In [15]:
# display tree
print(nodeL1.minmax())
print(nodeL2.minmax(), nodeL5.minmax())
print(nodeL3.minmax(), nodeL4.minmax(), nodeL6.minmax(), nodeL7.minmax())

L2,L5(ligne25)
L3,L4(ligne25)
L6,L7(ligne25)
39
L3,L4(ligne25)
L6,L7(ligne25)
39 12
44 39 17 12


In [16]:
# method alphabeta
print(nodeL1.alphabeta())
print(nodeL1.value)
print(nodeL2.value, nodeL5.value)
print(nodeL3.value, nodeL4.value, nodeL6.value, nodeL7.value)

L2,L5(ligne25)
L1 left > L2:L3,L4(ligne25)
L2 left > L3:value
L2 right > L4:value
L1 right > L5:L6,L7(ligne25)
L5 left > L6:value
L5 right pruning (ligne92)
39
39
39 17
44 39 17 12


In [17]:
# test class Tree
depth = 5
randMax = 10

In [18]:
# create tree
t = Tree(depth)
print(t)
print(t.set[depth-1][0])

Create tree
End create tree

0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	0,None,None	
1,(nodeL40,-10000,10000,None,None,2,True),(nodeL41,-10000,10000,None,None,5,True)	1,(nodeL42,-10000,10000,None,None,1,True),(nodeL43,-10000,10000,None,None,5,True)	1,(nodeL44,-10000,10000,None,None,0,True),(nodeL45,-10000,10000,None,None,5,True)	1,(nodeL46,-10000,10000,None,None,5,True),(nodeL47,-10000,10000,None,None,6,True)	1,(nodeL48,-10000,10000,None,None,8,True),(nodeL49,-10000,10000,None,None,4,True)	1,(nodeL410,-10000,10000,None,None,5,True),(nodeL411,-10000,10000,None,None,4,True)	1,(nodeL412,-10000,10000,None,None,7,True),(nodeL413,-10000,10000,None,None,1,True)	1,(nodeL414,-10000,10000,None,None,7,True),(nodeL415,-10000,10000,None,None,8,True)	
2,(nodeL30,-10000,10000,(nodeL40,-10000,10000,None,None,2,True),(nodeL41,-10000,10000,None,None,5,True),8,False),(n

In [19]:
print('Scanning tree')
print(t.set[depth-1][0].name, end=':')
t.set[depth-1][0].alphabeta()
#print('root value',t.set[depth-1][0].alphabeta())

Scanning tree
nodeL00:nodeL10,nodeL11(ligne25)
nodeL00 left > nodeL10:nodeL20,nodeL21(ligne25)
nodeL10 left > nodeL20:nodeL30,nodeL31(ligne25)
nodeL20 left > nodeL30:nodeL40,nodeL41(ligne25)
nodeL30 left > nodeL40:value
nodeL30 right > nodeL41:value
nodeL20 right > nodeL31:nodeL42,nodeL43(ligne25)
nodeL31 left > nodeL42:value
nodeL31 right pruning (ligne92)
nodeL10 right > nodeL21:nodeL32,nodeL33(ligne25)
nodeL21 left > nodeL32:nodeL44,nodeL45(ligne25)
nodeL32 left > nodeL44:value
nodeL32 right > nodeL45:value
nodeL21 right > nodeL33:nodeL46,nodeL47(ligne25)
nodeL33 left > nodeL46:value
nodeL33 right > nodeL47:value
nodeL00 right > nodeL11:nodeL22,nodeL23(ligne25)
nodeL11 left > nodeL22:nodeL34,nodeL35(ligne25)
nodeL22 left > nodeL34:nodeL48,nodeL49(ligne25)
nodeL34 left > nodeL48:value
nodeL34 right > nodeL49:value
nodeL22 right > nodeL35:nodeL410,nodeL411(ligne25)
nodeL35 left > nodeL410:value
nodeL35 right > nodeL411:value
nodeL11 right > nodeL23:nodeL36,nodeL37(ligne25)
nodeL23 lef

4

In [20]:
# draw inverse tree
t.drawTree()


Drawing tree
2,5,1,5,0,5,5,6,8,4,5,4,7,1,7,8,
2,1,0,2,4,4,1,4,
2,2,4,4,
2,4,
4,


In [21]:

for r in range(depth):
    print(['Node Min','Node Max'][t.set[r][0].isMax])
    for n in t.set[r]:
        if r:
            print((n.alpha,n.beta), end=';')
        else:
            print(n.value, end=',')
    print()

print(t.set[4][0].alphabeta())

Node Max
2,5,1,5,0,5,5,6,8,4,5,4,7,1,7,8,
Node Min
(-10000, 2);(2, 1);(-10000, 0);(0, 2);(2, 4);(4, 4);(2, 1);(2, 4);
Node Max
(2, 10000);(2, 2);(4, 10000);(4, 4);
Node Min
(-10000, 2);(2, 4);
Node Max
(4, 10000);
nodeL10,nodeL11(ligne25)
nodeL00 left > nodeL10:nodeL20,nodeL21(ligne25)
nodeL10 left > nodeL20:nodeL30,nodeL31(ligne25)
nodeL20 left > nodeL30:nodeL40,nodeL41(ligne25)
nodeL30 left > nodeL40:value
nodeL30 right pruning (ligne92)
nodeL20 right > nodeL31:nodeL42,nodeL43(ligne25)
nodeL31 left > nodeL42:value
nodeL31 right pruning (ligne92)
nodeL10 right pruning (ligne92)
nodeL00 right > nodeL11:nodeL22,nodeL23(ligne25)
nodeL11 left > nodeL22:nodeL34,nodeL35(ligne25)
nodeL22 left > nodeL34:nodeL48,nodeL49(ligne25)
nodeL34 left > nodeL48:value
nodeL34 right > nodeL49:value
nodeL22 right > nodeL35:nodeL410,nodeL411(ligne25)
nodeL35 left > nodeL410:value
nodeL35 right > nodeL411:value
nodeL11 right pruning (ligne92)
4
